# ML Model을 통한 수요예측

# 1. 필요 모듈/패키지 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)
import seaborn as sns
import os
import missingno as msno
import pickle
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

od_raw = pd.read_csv(r"orders.csv")
display(od_raw.head(), od_raw.info(), od_raw.isnull().sum())

# 2. 전처리

In [ ]:
od = od_raw.copy().iloc[:, :]

# 열이름 수정
od.columns = ["창고", "고객주문번호", "CJ주문번호", "주문유형", "주문날짜", "주문시간", "고객사코드", "주문금액", "품목순번",
             "품목코드", "브랜드", "품목수량", "품목금액", "수신여부", "주문생성시간", "택배구분", "상품주문번호", "중개업체주문번호",
             "접수여부", "배달예정점소코드", "배달예정사원코드", "터미널코드", "터미널소분류코드", "입력자ID", "입력일자",
             "입력시간", "권역구분", "배송처별주문분할여부", "송화인 광역주소", "송화인 지역주소", "수화인 광역주소", "수화인 지역주소",
             "주문월", "주문일", "주문요일", "주문시"]

# 주문유형에서 정상반출 제거

od = od[od["주문유형"] == 7]

In [ ]:
od.주문날짜 = od.주문날짜.astype("datetime64")
od.입력일자 = od.입력일자.astype("datetime64")

In [ ]:
od.info()

## 2-1. feature 추가
* 요일별 특성치
* 최근 물량 트렌드
* 캘린더 적용치 추가

In [ ]:
import datetime
# data_raw -> 주문날짜&주문시간대에 따른 전체날짜 데이터

od_kx = od[od["창고"] == "KX007"] # 곤지암 FC만 포함
data_raw = od_kx.groupby(["주문날짜", "주문시"])["품목수량"].sum().reset_index()  
data_raw = data_raw.append(pd.DataFrame(dict(zip(['주문날짜','주문시','품목수량'],
                                       [(pd.to_datetime('2021-06-28'),pd.to_datetime('2021-06-28')), 
                                        (4,5), (0,0)])))).sort_values(by = ["주문날짜", "주문시"]).reset_index(drop = True)
data_raw["주문요일"] = data_raw["주문날짜"].dt.dayofweek                           
data_raw["주문일"] = data_raw["주문날짜"].dt.day

for i in range(2, 15):
    data_raw[f"주문날짜+{i}"] = data_raw["주문날짜"].apply(lambda x : x + datetime.timedelta(days = i))

data_raw.head()

In [ ]:
target = data_raw[data_raw["주문날짜"] >= "2021-03-22"].loc[:, "품목수량"].reset_index(drop = True)

In [ ]:
############################# 최근물량 트렌드 #############################
# 7일전, 14일 전을 제외한 최근 2주간 시간
data = data_raw[data_raw["주문날짜"] >= "2021-03-22"].loc[:, "주문날짜" : "주문일"].drop("품목수량", axis = 1)
for i in range(2, 7):
    date_data = data_raw[[f"주문날짜+{i}", "주문시","품목수량"]].rename(columns = {f"주문날짜+{i}" : "주문날짜"})
    data = pd.merge(data, date_data, on = ["주문날짜", "주문시"], how = "left").rename(columns = {"품목수량" : f"{i}일전 품목수량"})

for i in range(8, 14):
    date_data = data_raw[[f"주문날짜+{i}", "주문시","품목수량"]].rename(columns = {f"주문날짜+{i}" : "주문날짜"})
    data = pd.merge(data, date_data, on = ["주문날짜", "주문시"], how = "left").rename(columns = {"품목수량" : f"{i}일전 품목수량"})
    


# 최근 2주동안 동요일 하루평균 수량
day_mean_7 = data_raw.groupby("주문날짜+7")["품목수량"].agg([("7일전 시간대별 평균수량",
                                              "mean")]).reset_index().rename(columns = {"주문날짜+7" : "주문날짜"})
day_mean_14 = data_raw.groupby("주문날짜+14")["품목수량"].agg([("14일전 시간대별 평균수량",
                                             "mean")]).reset_index().rename(columns = {"주문날짜+14" : "주문날짜"})

data = pd.merge(data, day_mean_7, how = "left")
data = pd.merge(data, day_mean_14, how = "left")

############################# 요일별 특성치 #############################

#14일 이전의 모든 동요일 동주문시에 대한 수량 평균
dd = []
for i in range(2424):
    df = data_raw[data_raw.주문날짜 < (data.iloc[i, 0] - datetime.timedelta(days = 14))]
    df = df[df.주문시 == data.iloc[i].주문시][df.주문요일 == data.iloc[i].주문요일].품목수량.mean()
    dd.append(df)
data["14일 이전 동요일 동주문시 평균수량"] = dd




############################# 캘린더 적용치 #############################

# LG생건X네이버 레드위크
def red_week(x):
    r1 = [pd.to_datetime(f'2021-03-{i}') for i in range(22,29)]
    r2 = [pd.to_datetime(f'2021-06-{i}') for i in range(7,16)]
    r = r1+r2
    if x in r:
        return 1
    else:
        return 0
data["레드위크"] = data["주문날짜"].map(red_week)    

# 네슬레 브랜드데이&구매왕 이벤트
def nestle(x):
    n = [pd.to_datetime("2021-03-22"), pd.to_datetime("2021-04-15"), 
         pd.to_datetime("2021-05-17"), pd.to_datetime("2021-06-17")]
    if x in n:
        return 1
    else:
        return 0
    
data["네슬레"] = data["주문날짜"].map(nestle)

# 뉴트리원 쇼핑라이브 경품 이벤트
def nutrione(x):
    n = [pd.to_datetime("2021-04-25"), pd.to_datetime("2021-04-30"), 
         pd.to_datetime("2021-05-16"), pd.to_datetime("2021-05-31"), 
         pd.to_datetime("2021-06-13"), pd.to_datetime("2021-06-27")]
    if x in n:
        return 1
    else:
        return 0
    
data["뉴트리원"] = data["주문날짜"].map(nutrione)

In [ ]:
data["target"] = target
data = data.drop("주문날짜", axis = 1)
data.fillna(0, inplace = True)
data.head()

## 2-2. 이상치 경계값으로 치환

In [ ]:
def get_outlier(df, column):
    fraud = df[column]
    qt_15 = np.percentile(fraud.values, 15)
    qt_85 = np.percentile(fraud.values, 85)
    
    iqr = qt_85 - qt_15
    iqr_w = iqr*1.5
    lowest_val = qt_15 - iqr_w
    highest_val = qt_85 + iqr_w
    
    low_index = fraud[(fraud < lowest_val)].index
    low_qt = qt_15
    
    high_index = fraud[(fraud >highest_val)].index
    high_qt = qt_85
    
    
    return low_index, high_index, low_qt, high_qt

# feature
for i in data.columns[3:-4]:
    low_index, high_index, low_qt, high_qt = get_outlier(data, i)
    data.loc[low_index, i] = low_qt
    data.loc[high_index, i] = high_qt

In [ ]:
plt.figure(figsize = (20, 12))
data.iloc[:, 3:-4].boxplot()
plt.xticks(fontsize = 15, rotation = 20)
plt.ylim(0, 8500)
plt.show()

## 2-3. feature별 상관관계 확인

In [ ]:
plt.figure(figsize = (20, 12))
sns.heatmap(data.corr(), annot = True)
plt.show()

# 3. Modeling

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

## 3-1  전통적 방식의 학습 -> 개별 모델 최적화 후 예측

In [ ]:
## train -> 3, 4, 5월 데이터
## validation  -> 6/1 ~ 6/15
## test -> 6/16 ~ 6/30

X_train = data.iloc[:1704].drop("target", axis = 1)
X_val = data.iloc[1704:2064].drop("target", axis = 1)
X_test = data.iloc[2064:].drop("target", axis = 1)

y_train = data.iloc[:1704].target
y_val = data.iloc[1704:2064].target
y_test = data.iloc[2064:].target

### 베이지안 최적화

In [ ]:
pbounds = {
    'max_depth':(2, 128),
    'n_estimators':(10,500)
}

def xgb_opt(max_depth, n_estimators):
    params = {
      'max_depth':int(round(max_depth)),
      'n_estimators':int(round(n_estimators))
     }
    xgb = XGBRegressor(**params, random_state=0, n_jobs=-1)
    score = cross_val_score(xgb, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    return -np.sqrt(-np.mean(score))

BO_xgb = BayesianOptimization(f=xgb_opt, pbounds=pbounds, random_state=0)
BO_xgb.maximize(init_points=50, n_iter=50)

In [ ]:
pbounds = {
    'max_depth':(2, 128),
    'n_estimators':(10,500)
}

def et_opt(max_depth, n_estimators):
    params = {
      'max_depth':int(round(max_depth)),
      'n_estimators':int(round(n_estimators))
   }
    et = ExtraTreesRegressor(**params, random_state=0, n_jobs=-1)
    score = cross_val_score(et, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    return -np.sqrt(-np.mean(score))

BO_et = BayesianOptimization(f=et_opt, pbounds=pbounds, random_state=0)
BO_et.maximize(init_points=50, n_iter=50)

In [ ]:
pbounds = {
    'num_leaves':(2,128),
    'max_depth':(2, 128),
    'learning_rate':(0.0000001,10),
    'n_estimators':(10,500),
    'reg_alpha':(0.0000000001, 10),
    'reg_lambda':(0.00000000001, 10)
}

def lgbm_opt(num_leaves, max_depth, learning_rate, n_estimators, reg_alpha, reg_lambda):
    params = {
      'num_leaves':int(round(num_leaves)),
      'max_depth':int(round(max_depth)),
      'learning_rate':learning_rate,
      'n_estimators':int(round(n_estimators)),
      'reg_alpha':reg_alpha,
      'reg_lambda':reg_lambda
  }
    lgbm = LGBMRegressor(**params, random_state=0, n_jobs=-1)
    score = cross_val_score(lgbm, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    return -np.sqrt(-np.mean(score))

BO_lgbm = BayesianOptimization(f=lgbm_opt, pbounds=pbounds, random_state=0)
BO_lgbm.maximize(init_points=50, n_iter=50)

In [ ]:
pbounds = {
    'max_depth':(2, 128),
    'n_estimators':(10,500)
}

def rf_opt(max_depth, n_estimators):
    params = {
      'max_depth':int(round(max_depth)),
      'n_estimators':int(round(n_estimators))
  }
    rf = RandomForestRegressor(**params, random_state=0, n_jobs=-1)
    score = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    return -np.sqrt(-np.mean(score))

BO_rf = BayesianOptimization(f=rf_opt, pbounds=pbounds, random_state=0)
BO_rf.maximize(init_points=50, n_iter=50)

In [ ]:
max_params_xgb = BO_rf.max['params']
max_params_xgb['max_depth'] = int(round(max_params_xgb['max_depth']))
max_params_xgb['n_estimators'] = int(round(max_params_xgb['n_estimators']))
max_params_xgb

In [ ]:
max_params_et = BO_et.max['params']
max_params_et['max_depth'] = int(round(max_params_et['max_depth']))
max_params_et['n_estimators'] = int(round(max_params_et['n_estimators']))
max_params_et

In [ ]:
max_params_lgbm = BO_lgbm.max['params']
max_params_lgbm['max_depth'] = int(round(max_params_lgbm['max_depth']))
max_params_lgbm['n_estimators'] = int(round(max_params_lgbm['n_estimators']))
max_params_lgbm['num_leaves'] = int(round(max_params_lgbm['num_leaves']))
max_params_lgbm

In [ ]:
max_params_rf = BO_rf.max['params']
max_params_rf['max_depth'] = int(round(max_params_rf['max_depth']))
max_params_rf['n_estimators'] = int(round(max_params_rf['n_estimators']))
max_params_rf

In [ ]:
xgb = XGBRegressor(**max_params_xgb, random_state=0, n_jobs=-1)
et = ExtraTreesRegressor(**max_params_et, random_state=0, n_jobs=-1)
lgbm = LGBMRegressor(**max_params_lgbm, random_state=0, n_jobs=-1)
cat = CatBoostRegressor(random_state=0)
rf = RandomForestRegressor(**max_params_rf, random_state=0, n_jobs=-1)

### 학습

In [ ]:
xgb.fit(X_train, y_train)
et.fit(X_train, y_train)
lgbm.fit(X_train, y_train)
cat.fit(X_train, y_train)
rf.fit(X_train, y_train)

In [ ]:
models = [xgb, et, lgbm, cat, rf]
for model in tqdm(models):
    with open(f"{str(model).split('(')[0].split('.')[0].replace('<','')}.pkl", 'wb') as f:
        pickle.dump(model, f)

In [ ]:
xgb = pickle.load(open("XGBRegressor.pkl", 'rb'))
et = pickle.load(open("ExtraTreesRegressor.pkl", 'rb'))
lgbm = pickle.load(open("LGBMRegressor.pkl", 'rb'))
cat = pickle.load(open("catboost.pkl", 'rb'))
rf = pickle.load(open("RandomForestRegressor.pkl", 'rb'))

### 모델별 검증/평가데이터 점수

In [ ]:
models = [xgb, et, lgbm, cat, rf]

for model in models:
    print(model.__class__.__name__)
    print("검증데이터 RMSE : ",np.sqrt(mean_squared_error(y_val, model.predict(X_val))))
    print("평가데이터 RMSE : ",np.sqrt(mean_squared_error(y_test, model.predict(X_test))))
    print("\n")

## 3-2. OOF(Out-of-Fold)

In [ ]:
# train -> 3, 4, 5월

X = data.iloc[:1704].drop("target", axis = 1)
X_test = data.iloc[2064:].drop("target", axis = 1)

y = data.iloc[:1704].target
y_test = data.iloc[2064:].target

#### XGBRegressor

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

model = xgb
xgb_pred = np.zeros((X_test.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)

    rmse = np.sqrt(mean_squared_error(val_y, pred))
    rmse_list.append(rmse)
    
    sub_pred = np.array(model.predict(X_test)) / 5
    xgb_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 RMSE는 {np.mean(rmse_list)}')

#### ExtraTreesRegressor

In [ ]:
model = et
et_pred = np.zeros((X_test.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)

    rmse = np.sqrt(mean_squared_error(val_y, pred))
    rmse_list.append(rmse)
    
    sub_pred = np.array(model.predict(X_test)) / 5
    et_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 RMSE는 {np.mean(rmse_list)}')

#### LGBMRegressor

In [ ]:
model = lgbm
lgbm_pred = np.zeros((X_test.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)

    rmse = np.sqrt(mean_squared_error(val_y, pred))
    rmse_list.append(rmse)
    
    sub_pred = np.array(model.predict(X_test)) / 5
    lgbm_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 RMSE는 {np.mean(rmse_list)}')

#### CatBoostRegressor

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

model = cat
cat_pred = np.zeros((X_test.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)

    rmse = np.sqrt(mean_squared_error(val_y, pred))
    rmse_list.append(rmse)
    
    sub_pred = np.array(model.predict(X_test)) / 5
    cat_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 RMSE는 {np.mean(rmse_list)}')

In [ ]:
data = pd.Series(cat.feature_importances_, index = X.columns).sort_values(ascending= False)

plt.figure(figsize = (20, 12))
data.plot.bar()
plt.xticks(rotation = 60)
plt.title("OOF-catboost 모델 피쳐 중요도")
plt.grid()

for i, imp in enumerate(data):
    plt.text(i, imp+0.1, f"{np.round(imp, 2)}", fontsize = 15, ha = "center")
plt.show()

In [ ]:
for i in pd.Series(cat.feature_importances_, index = X.columns).sort_values(ascending= False).values:
    print(i)

#### RandomForestRegressor

In [ ]:
model = rf
rf_pred = np.zeros((X_test.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)

    rmse = np.sqrt(mean_squared_error(val_y, pred))
    rmse_list.append(rmse)
    
    sub_pred =np.array(model.predict(X_test)) / 5
    rf_pred += sub_pred
print(f'{model.__class__.__name__}의 5fold 평균 RMSE는 {np.mean(rmse_list)}')

## Final OOF RMSE

In [ ]:
final_pred = (xgb_pred + et_pred + lgbm_pred + cat_pred + rf_pred) / 5
print(f"Final blending model RMSE : {np.sqrt(mean_squared_error(y_test, final_pred))}")

In [ ]:
np.sqrt((mean_squared_error(y_test, cat_pred)))

In [ ]:
np.sqrt((mean_squared_error(y_test, lgbm_pred)))

In [ ]:
np.sqrt((mean_squared_error(y_test, xgb_pred)))

In [ ]:
np.sqrt((mean_squared_error(y_test, et_pred)))

In [ ]:
np.sqrt((mean_squared_error(y_test, rf_pred)))

In [ ]:
data_raw[data_raw.주문날짜 > pd.to_datetime("2021-06-15")].주문날짜.astype("str").values

In [ ]:
len(cat_pred)

## 최고성능 모델 예측값 시각화
* OOF CatBoostRegressor  모델

In [ ]:
y_test = data.iloc[2064:].target
final = pd.DataFrame()
final["real"] = y_test
final["preds"] = cat_pred
final.index = data_raw[data_raw.주문날짜 > pd.to_datetime("2021-06-15")].주문날짜.astype("str").values
final

In [ ]:
# CatBoostRegressor모델 예측 시각화
rmse = np.sqrt(mean_squared_error(y_test, cat_pred))
x = ["2021-06-01", "2021-06-04", "2021-06-09", "2021-06-13", "2021-06-17", "2021-06-21", "2021-06-25", "2021-06-29"]
figure, ax = plt.subplots(figsize = (20, 12))
plt.title("OOF - CatBoostRegressor", fontsize = 40)
final.preds.plot(c = "black", label = "preds")
final.real.plot(c = "pink", label = "real")
plt.ylabel("")
plt.legend(fontsize = 30)
plt.grid()
plt.show()

In [ ]:
ml_pred = pd.Series(cat_pred)
ml_pred.to_csv("results_ml.csv", index = False)